# Project: Segmenting and Clustering Neighborhoods in UK



## Introduction

One of the issues when parents choose whether to settle in a district or area to live is whether a location has good schools or not and what are the surrounding neighbourhoods. Are the surroundings suitable for children? Take for an example, if a family settle in Bristol, most likely this family has made the decision to retire in this near-seashore city. In Bristol, when looking up its neighbourhood venues in Foursquare, many of them are bars, which is very much different from that of Oxford. Therefore, finding out schools and its surrounding could help parents making sensible choices when choosing a place to settle or for their children to study.

## Data
The data came from the open data initiatives of UK government.

Schools data were downloaded from https://ea-edubase-api-prod.azurewebsites.net/edubase/downloads/public/edubasealldata20210201.csv, provided by Gov.uk. These data contained information about a school: its location, its establishment group, its religion, its Ofsted Rating, etc. There is no latitude and longitude data.

To get the latitude, longitude and nearby postcodes of a postcode, this API is used : https://api.postcodes.io/.

Venues data are from Foursquare.

## Methodology
First, a complete list of postcodes with NHS info and a complete list of education establishment were downloaded from gov.uk. Initial exploration was done to understand what data were available for investigation.
The datasets were merged and postcodes with no schools were removed. For the purpose of this exercise, the NHS info was not used for further investigation.
Since there is no latitude and longitude info in both data set, the data was obtained from an online API for each postcode.
Next we drill into one of the cities to explore the rating and number of schools inside the city and create a visual map for further exploration.
The neighbourhoods were identified using an online API. As the results returned were in postcodes, they were converted to latitude and longitude before exploring further with Foursquare. Afterwards, neighbourhoods were explored.


## Result
Using one of the postcodes in City of Bristol as example, the result showed that the most common venue in the neighbourhoods are Pub, clothing stores and Cafe. This reflects the reality that Bristol is a travel city, good for retirement and vacation.
However, the concentration of schools is quite different from that of Oxford. The result could serve as an initial supporting evidence for selecting an area to settle or study.

## Discussion
This exercise is a framework for analysis. For further improvement, it would be interesting to see any spending figures or income figures added to the dataframe for analysis. For example, is there any relationship between income, study area and transport availability? What is the best place to study if a person do not want to own a car? This framework can be extended further to answer other interesting questions.

## This following scripts import data from web and store it in a dataframe

In [2]:
# import libraries
import pandas as pd
import numpy as np

## Import data of UK from data downloaded from Gov.UK
## A set of postcodes in CSV format containing location info of respective postcodes

In [3]:
import os

# Getting the current work directory (cwd)
datadir = '/codepo_gb/Data/CSV/'
thisdir = os.getcwd() + datadir

dfs = []

# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir):
    for file in f:
        if file.endswith(".csv"):
            print(os.path.join(r, file))
            currentcsv = pd.read_csv('.' + datadir + file, header=None)
            dfs.append(currentcsv)

# Concatenate all data into one DataFrame
locationdf = pd.concat(dfs)
locationdf.columns = ['Postcode','Positional_quality_indicator','Eastings','Northings','Country_code','NHS_regional_HA_code','NHS_HA_code','Admin_county_code','Admin_district_code','Admin_ward_code']

print(locationdf.shape)

/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/cr.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/w.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/ba.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/wa.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/wv.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/al.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/bb.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/cf.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/wc.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/ze.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/tn.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/b.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/nw.csv
/Users/yckan/Documents/github/Coursera_C

/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/ta.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/ch.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/bl.csv
/Users/yckan/Documents/github/Coursera_Capstone/codepo_gb/Data/CSV/m.csv
(1711275, 10)


In [4]:
locationdf.head()

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,CR0 0AA,10,539052,162028,E92000001,E19000003,E18000007,NaN,E09000008,E05011471
1,CR0 0AB,10,539266,161728,E92000001,E19000003,E18000007,NaN,E09000008,E05011471
2,CR0 0AD,10,539327,161477,E92000001,E19000003,E18000007,NaN,E09000008,E05011471
3,CR0 0AE,10,539086,162071,E92000001,E19000003,E18000007,NaN,E09000008,E05011471
4,CR0 0AF,10,539322,161810,E92000001,E19000003,E18000007,NaN,E09000008,E05011471


## A set of schools related information is downloaded from the web in CSV format
## School info is loaded into a dataframe

In [5]:
schoolsFile = './codepo_gb/Data/edubasealldata20210117-utf8.csv'
schoolsdf = pd.read_csv(schoolsFile)
schoolsdf.head()


/Users/yckan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (48,50,51,72,73,96,124,130) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,URN,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentTypeGroup (code),EstablishmentTypeGroup (name),EstablishmentStatus (code),...,UPRN,SiteName,QABName (code),QABName (name),EstablishmentAccredited (code),EstablishmentAccredited (name),QABReport,CHNumber,MSOA (code),LSOA (code)
0,100000,201,City of London,3614.0,The Aldgate School,2,Voluntary aided school,4,Local authority maintained schools,1,...,2.000001e+11,NaN,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02000001,E01032739
1,100001,201,City of London,6005.0,City of London School for Girls,11,Other independent school,3,Independent schools,1,...,2.000001e+11,NaN,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02000001,E01000002
2,100002,201,City of London,6006.0,St Paul's Cathedral School,11,Other independent school,3,Independent schools,1,...,2.000001e+11,NaN,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02000001,E01032739
3,100003,201,City of London,6007.0,City of London School,11,Other independent school,3,Independent schools,1,...,1.000229e+11,NaN,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02000001,E01032739
4,100004,202,Camden,1045.0,Sherborne Nursery School,15,Local authority nursery school,4,Local authority maintained schools,2,...,2.001639e+08,NaN,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02000172,E01000901


## Let's take a look at the column names and shape of the dataframe

In [6]:
for items in schoolsdf.columns:
    print (items)

URN
LA (code)
LA (name)
EstablishmentNumber
EstablishmentName
TypeOfEstablishment (code)
TypeOfEstablishment (name)
EstablishmentTypeGroup (code)
EstablishmentTypeGroup (name)
EstablishmentStatus (code)
EstablishmentStatus (name)
ReasonEstablishmentOpened (code)
ReasonEstablishmentOpened (name)
OpenDate
ReasonEstablishmentClosed (code)
ReasonEstablishmentClosed (name)
CloseDate
PhaseOfEducation (code)
PhaseOfEducation (name)
StatutoryLowAge
StatutoryHighAge
Boarders (code)
Boarders (name)
NurseryProvision (name)
OfficialSixthForm (code)
OfficialSixthForm (name)
Gender (code)
Gender (name)
ReligiousCharacter (code)
ReligiousCharacter (name)
ReligiousEthos (name)
Diocese (code)
Diocese (name)
AdmissionsPolicy (code)
AdmissionsPolicy (name)
SchoolCapacity
SpecialClasses (code)
SpecialClasses (name)
CensusDate
NumberOfPupils
NumberOfBoys
NumberOfGirls
PercentageFSM
TrustSchoolFlag (code)
TrustSchoolFlag (name)
Trusts (code)
Trusts (name)
SchoolSponsorFlag (name)
SchoolSponsors (name)
Feder

In [7]:
schoolsdf.shape

(48858, 139)

## We select a subset from the dataframe and see what is the information of a school.

In [8]:
schools_subsetdf = schoolsdf[['Postcode','URN','LA (name)','EstablishmentName','TypeOfEstablishment (name)','EstablishmentTypeGroup (name)','PhaseOfEducation (name)','Gender (name)','ReligiousCharacter (name)','OfstedRating (name)','Easting','Northing']]
schools_subsetdf.head()

,Postcode,URN,LA (name),EstablishmentName,TypeOfEstablishment (name),EstablishmentTypeGroup (name),PhaseOfEducation (name),Gender (name),ReligiousCharacter (name),OfstedRating (name),Easting,Northing
0,EC3A 5DE,100000,City of London,The Aldgate School,Voluntary aided school,Local authority maintained schools,Primary,Mixed,Church of England,Outstanding,533498.0,181201.0
1,EC2Y 8BB,100001,City of London,City of London School for Girls,Other independent school,Independent schools,Not applicable,Girls,None,NaN,532301.0,181746.0
2,EC4M 9AD,100002,City of London,St Paul's Cathedral School,Other independent school,Independent schools,Not applicable,Mixed,Church of England,NaN,532160.0,181151.0
3,EC4V 3AL,100003,City of London,City of London School,Other independent school,Independent schools,Not applicable,Boys,None,NaN,531981.0,180844.0
4,NW5 4LP,100004,Camden,Sherborne Nursery School,Local authority nursery school,Local authority maintained schools,Nursery,Mixed,Does not apply,NaN,528515.0,184869.0


## We merge the location dataframe and schools dataframe

In [9]:
# merge dataframes
new_df = pd.merge(locationdf, schoolsdf,  how='left', left_on=['Postcode'], right_on = ['Postcode'])

## See what types of schools are located near Oxford and what are their ratings

In [11]:
ox = new_df[new_df['Town']=='Oxford']
ox[['County (name)','Town','Postcode','EstablishmentName','EstablishmentTypeGroup (name)','OfstedRating (name)']].groupby(['Town','EstablishmentTypeGroup (name)','OfstedRating (name)']).count()

County (name)  \
Town   EstablishmentTypeGroup (name)      OfstedRating (name)                   
Oxford Academies                          Good                             10   
                                          Outstanding                       1   
                                          Requires improvement              3   
                                          Special Measures                  4   
       Colleges                           Good                              1   
       Free Schools                       Good                              1   
       Independent schools                Good                              3   
       Local authority maintained schools Good                             21   
                                          Outstanding                       4   
                                          Requires improvement              1   
                                          Serious Weaknesses                2   
                                          Special Measures                  2   
       Special schools                    Good                              1   
                                          Outstanding                       2   
                                          Special Measures                  1   
       Universities                       Good                              1   

                                                                Postcode  \
Town   EstablishmentTypeGroup (name)      OfstedRating (name)              
Oxford Academies                          Good                        11   
                                          Outstanding                  1   
                                          Requires improvement         3   
                                          Special Measures             4   
       Colleges                           Good                         1   
       Free Schools                       Good                         1   
       Independent schools                Good                         3   
       Local authority maintained schools Good                        21   
                                          Outstanding                  4   
                                          Requires improvement         1   
                                          Serious Weaknesses           2   
                                          Special Measures             2   
       Special schools                    Good                         2   
                                          Outstanding                  2   
                                          Special Measures             1   
       Universities                       Good                         1   

                                                                EstablishmentName  
Town   EstablishmentTypeGroup (name)      OfstedRating (name)                      
Oxford Academies                          Good                                 11  
                                          Outstanding                           1  
                                          Requires improvement                  3  
                                          Special Measures                      4  
       Colleges                           Good                                  1  
       Free Schools                       Good                                  1  
       Independent schools                Good                                  3  
       Local authority maintained schools Good                                 21  
                                          Outstanding                           4  
                                          Requires improvement                  1  
                                          Serious Weaknesses                    2  
                                          Special Measures                      2  
       Special schools                    Good              

## The shape of the merged dataframe that contains schools and location info

In [12]:
new_df.shape

(1719836, 148)

## Print the column names for use later

In [13]:
for items in new_df.columns:
    print (items)

Postcode
Positional_quality_indicator
Eastings
Northings
Country_code
NHS_regional_HA_code
NHS_HA_code
Admin_county_code
Admin_district_code
Admin_ward_code
URN
LA (code)
LA (name)
EstablishmentNumber
EstablishmentName
TypeOfEstablishment (code)
TypeOfEstablishment (name)
EstablishmentTypeGroup (code)
EstablishmentTypeGroup (name)
EstablishmentStatus (code)
EstablishmentStatus (name)
ReasonEstablishmentOpened (code)
ReasonEstablishmentOpened (name)
OpenDate
ReasonEstablishmentClosed (code)
ReasonEstablishmentClosed (name)
CloseDate
PhaseOfEducation (code)
PhaseOfEducation (name)
StatutoryLowAge
StatutoryHighAge
Boarders (code)
Boarders (name)
NurseryProvision (name)
OfficialSixthForm (code)
OfficialSixthForm (name)
Gender (code)
Gender (name)
ReligiousCharacter (code)
ReligiousCharacter (name)
ReligiousEthos (name)
Diocese (code)
Diocese (name)
AdmissionsPolicy (code)
AdmissionsPolicy (name)
SchoolCapacity
SpecialClasses (code)
SpecialClasses (name)
CensusDate
NumberOfPupils
NumberOfBo

## Remove rows that have no schools (no EstablishmentName)

In [14]:
new_df.dropna(subset=['EstablishmentName'], inplace=True)
new_df.shape

(24019, 148)

## The dataframe became much smaller to handle

## There could be more than 1 schools in a postcode
## Get the unique postcodes

In [15]:
print(new_df.Postcode.unique().shape)

(15458,)


## Types and count of schools with OfstedRating
## Parents can further drill into the grouping to get the list of schools in different postcodes

## Group schools by type and Ofsted rating and see how many schools are there for each type

In [16]:
print(new_df.groupby(['EstablishmentTypeGroup (name)','OfstedRating (name)']).size().reset_index(name='counts'))

         EstablishmentTypeGroup (name)   OfstedRating (name)  counts
0                            Academies                  Good    2093
1                            Academies           Outstanding     410
2                            Academies  Requires improvement     474
3                            Academies    Serious Weaknesses      41
4                            Academies      Special Measures      82
5                             Colleges                  Good      87
6                             Colleges            Inadequate       2
7                             Colleges           Outstanding      24
8                             Colleges  Requires improvement      27
9                         Free Schools                  Good     122
10                        Free Schools           Outstanding      65
11                        Free Schools  Requires improvement      30
12                        Free Schools    Serious Weaknesses       5
13                        Free Sch

## Show the data of schools located in a certain county, town and postcodes

In [17]:
new_df_subset = new_df[['EstablishmentTypeGroup (name)','EstablishmentName','OfstedRating (name)','County (name)','Town','Postcode']]
new_df_subset.head()

,EstablishmentTypeGroup (name),EstablishmentName,OfstedRating (name),County (name),Town,Postcode
6,Local authority maintained schools,Fairchildes Primary School,Outstanding,Surrey,Croydon,CR0 0AH
7,Local authority maintained schools,Addington High School,NaN,Surrey,Croydon,CR0 0AH
8,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH
9,Academies,Fairchildes Primary School,NaN,Surrey,Croydon,CR0 0AH
10,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH


In [18]:
new_df_subset.shape

(24019, 6)

## Group the schools by Town and see how many schools each has

In [19]:
new_df_subset.groupby(['Town']).head()

,EstablishmentTypeGroup (name),EstablishmentName,OfstedRating (name),County (name),Town,Postcode
6,Local authority maintained schools,Fairchildes Primary School,Outstanding,Surrey,Croydon,CR0 0AH
7,Local authority maintained schools,Addington High School,NaN,Surrey,Croydon,CR0 0AH
8,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH
9,Academies,Fairchildes Primary School,NaN,Surrey,Croydon,CR0 0AH
10,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH
...,...,...,...,...,...,...
1706612,Local authority maintained schools,Lime Tree Primary School,Outstanding,Cheshire,Sale,M33 2RP
1708863,Academies,Linden Road Academy and Hearing Impaired Base,Good,NaN,Tameside,M34 6EF
1712178,Independent schools,Abbotsford Preparatory School,NaN,Lancashire,M41 5PR,M41 5PR
1714596,Local authority maintained schools,The Ark,NaN,Greater Manchester,Whitefield,M45 8NH


## Get the top 20 towns with the most schools

In [20]:
new_df_subset[['Town', 'Postcode', 'EstablishmentName']].groupby(['Town','Postcode']).count().sort_values(by='EstablishmentName', ascending=False).head(20)

,,EstablishmentName
Town,Postcode,
London,E14 9TS,16
Manchester,M29 8BS,14
Tyldesley,M29 8BS,11
Newcastle-upon-Tyne,NE4 8XJ,8
Peterborough,PE4 6HX,8
London,N16 5RP,8
Blackpool,FY3 9JL,8
Middlesbrough,TS3 8RD,7
Skelmersdale,WN8 8EH,7


In [21]:
new_df_subset[['Town', 'Postcode', 'EstablishmentName','OfstedRating (name)']].groupby(['Town','OfstedRating (name)','Postcode']).count().head(50)

EstablishmentName
Town                OfstedRating (name)  Postcode                   
5 Cliftonville Road Outstanding          NN1 5DG                   1
Abberley            Good                 WR6 6AA                   1
Abbey Wood          Good                 SE2 9TA                   1
Abbots Langley      Good                 WD5 0BQ                   1
                                         WD5 0HW                   1
                                         WD5 0LG                   1
                    Outstanding          WD5 0BU                   1
                    Special Measures     WD5 0RD                   1
Accrington          Good                 BB5 0JD                   1
                                         BB5 1DW                   1
                                         BB5 1ST                   1
                                         BB5 2AQ                   1
                                         BB5 2HS                   1
                                         BB5 2LH                   1
                                         BB5 2QY                   1
                                         BB5 2RQ                   1
                                         BB5 2SW                   1
                                         BB5 3AA                   1
                                         BB5 3BT                   1
                                         BB5 3DA                   1
                                         BB5 3DD                   1
                                         BB5 3JG                   1
                                         BB5 3LG                   1
                                         BB5 3SQ                   1
                                         BB5 4AY                   1
                                         BB5 4DN                   1
                                         BB5 4FF                   1
                                         BB5 5HT                   1
                                         BB5 5RJ                   1
                                         BB5 5UH                   1
                                         BB5 6AD                   1
                                         BB5 6LR                   1
                    Outstanding          BB5 0LD                   1
                                         BB5 0NN                   1
                                         BB5 3BE                   1
                                         BB5 4QG                   1
                                         BB5 5HT                   1
                                         BB5 5NH                   1
                                         BB5 6QR                   1
                    Requires improvement BB5 0LU                   1
                                         BB5 0NW                   1
                                         BB5 2AN                   1
                                         BB5 3EA                   1
                                         BB5 4HG                   1
Acklam              Outstanding          TS5 8RZ                   1
Acton Beauchamp     Requires improvement WR6 5AH                   1
Adlington           Good                 PR6 9NN                   1
Alcester            Good                 B49 5HN                   1
                                         B49 5NT                   1
                                         B49 6AG                   1

## Drill further into a Town and see the rating of the schools

In [182]:
new_df_subset1=new_df_subset[new_df_subset['Town']=="Oxford"].sort_values(by='OfstedRating (name)')
new_df_subset1

,EstablishmentTypeGroup (name),EstablishmentName,OfstedRating (name),County (name),Town,Postcode
1315119,Independent schools,Carfax College,Good,Oxfordshire,Oxford,OX1 2EP
1331228,Local authority maintained schools,Church Cowley St James Church of England Prima...,Good,Oxfordshire,Oxford,OX4 3QH
1329481,Academies,Cheney School,Good,Oxfordshire,Oxford,OX3 7QH
1329525,Local authority maintained schools,"St Joseph's Catholic Primary School, Oxford",Good,Oxfordshire,Oxford,OX3 7SX
1329598,Academies,"Endeavour Academy, Oxford",Good,Oxfordshire,Oxford,OX3 8DD
1331087,Local authority maintained schools,Comper Foundation Stage School,Good,Oxfordshire,Oxford,OX4 3AJ
1331066,Independent schools,Kings Education (Oxford),Good,Oxfordshire,Oxford,OX4 2UJ
1331020,Local authority maintained schools,St Francis Church of England Primary School,Good,Oxfordshire,Oxford,OX4 2QT
1331780,Special schools,Mabel Prichard School,Good,NaN,Oxford,OX4 6SB
1329752,Local authority maintained schools,Wood Farm Primary School,Good,Oxfordshire,Oxford,OX3 8QQ


## The following APIs return the information of a location using postcode
### Use OX1 2EP, the first school with 'Good' ratings, as an example to retrieve the info

In [183]:
from requests.utils import requote_uri

# lookup longitude and latitude
postcode='OX1 2EP'

# Webpage url                                                                                                               
url = 'https://api.postcodes.io/postcodes/' + postcode
urlencoded = requote_uri(url)

# Extract tables
dfs = pd.read_json(urlencoded)
dfs = dfs.T
print(dfs)


       admin_county admin_district         admin_ward              ccg  \
status          200            200                200              200   
result  Oxfordshire         Oxford  Jericho and Osney  NHS Oxfordshire   

                      ced                                              codes  \
status                200                                                200   
result  Jericho and Osney  {'admin_district': 'E07000178', 'admin_county'...   

        country eastings european_electoral_region incode latitude longitude  \
status      200      200                       200    200      200       200   
result  England   450758                South East    2EP  51.7535  -1.26606   

               lsoa        msoa         nhs_ha northings         nuts outcode  \
status          200         200            200       200          200     200   
result  Oxford 009D  Oxford 009  South Central    206340  Oxfordshire     OX1   

                         parish parliamentary_const

## The following unique postcodes list was prepared for using a batch API to get information of all unique postcodes in UK including latitude and longitude which can be used to get a visual map of the schools location

In [24]:
print (new_df.Postcode.unique().shape[0])

s = new_df.Postcode.unique().shape[0]
q = new_df.Postcode.unique().shape[0]//100
r = new_df.Postcode.unique().shape[0]%100

for i in np.arange(0,s,100):
    j=i+100
    if i == q*100:
        j=i+r+1
        print ('Last batch is ', j)
    data = {'postcodes': new_df.Postcode.unique()[i:j].tolist() }



15458
Last batch is  15459


In [25]:
# get longitude and latitude
import requests
from requests.exceptions import HTTPError
import json
import time

# data prep
#data = {'postcodes': new_df.Postcode.unique()[0:100].tolist() }
    
url = "https://api.postcodes.io/postcodes"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

s = new_df.Postcode.unique().shape[0]
q = new_df.Postcode.unique().shape[0]//100
r = new_df.Postcode.unique().shape[0]%100

def getData(listToProcess):
    # execute
    try:
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()
        # access Json content
        jsonResponse = response.json()
        #print("Entire JSON response")
        #print(jsonResponse)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')       
    return jsonResponse

resultSet = []

for i in np.arange(0,s,100):
    j=i+100
    if i == q*100:
        j=i+r+1
        print ('Last batch is ', j)
    print ('Batch:',i,' to ',j)
    data = {'postcodes': new_df.Postcode.unique()[i:j].tolist() }
    
    # wait 1s before query
    time.sleep(1)
    returnedData = pd.DataFrame.from_dict(getData(data), orient='index')
    returnedData.reset_index(level=0, inplace=True)
    print('Head(5):',returnedData.head(5))
    # append result to resultSet                                      
    resultSet.append(returnedData)



Batch: 0  to  100
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'CR0 0EG', 'result': {'postcode': '...
Batch: 100  to  200
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'CR3 6AF', 'result': {'postcode': '...
Batch: 200  to  300
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'W10 4DQ', 'result': {'postcode': '...
Batch: 300  to  400
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'W1J 7NL', 'result': {'postcode': '...
Batch: 400  to  500
Head(5):     index                                                  0
0  status                                                200
1  

Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'E17 5HU', 'result': {'postcode': '...
Batch: 4000  to  4100
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'BS5 7SY', 'result': {'postcode': '...
Batch: 4100  to  4200
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'WR3 8LE', 'result': {'postcode': '...
Batch: 4200  to  4300
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'YO8 8NL', 'result': {'postcode': '...
Batch: 4300  to  4400
Head(5):     index                                                  0
0  status                                                200
1  result  [{

Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'DE7 6FB', 'result': {'postcode': '...
Batch: 7900  to  8000
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'HD7 5SP', 'result': {'postcode': '...
Batch: 8000  to  8100
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'SY4 3RP', 'result': {'postcode': '...
Batch: 8100  to  8200
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'SN2 7HJ', 'result': {'postcode': '...
Batch: 8200  to  8300
Head(5):     index                                                  0
0  status                                                200
1  result  [{

Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'CM7 4RN', 'result': {'postcode': '...
Batch: 11800  to  11900
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'LU3 2NX', 'result': {'postcode': '...
Batch: 11900  to  12000
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'OL1 3SQ', 'result': {'postcode': '...
Batch: 12000  to  12100
Head(5):     index                                                  0
0  status                                                200
1  result  [{'query': 'OL7 0BJ', 'result': {'postcode': '...
Batch: 12100  to  12200
Head(5):     index                                                  0
0  status                                                200
1  re

## Convert the results into a big dataframe
## write the results to a CSV file

In [26]:
print(len(resultSet))

postcodeWithSchools = pd.DataFrame(columns=['query','result'])
for i in np.arange(0,len(resultSet)):
    df1 = pd.DataFrame(resultSet[i].iloc[1,1])
    postcodeWithSchools = pd.concat([postcodeWithSchools,df1])
    
print(postcodeWithSchools.head())
postcodeWithSchools.shape


155
     query                                             result
0  CR0 0EG  {'postcode': 'CR0 0EG', 'quality': 1, 'easting...
1  CR0 0PA  {'postcode': 'CR0 0PA', 'quality': 1, 'easting...
2  CR0 1EQ  {'postcode': 'CR0 1EQ', 'quality': 1, 'easting...
3  CR0 0AH  {'postcode': 'CR0 0AH', 'quality': 1, 'easting...
4  CR0 1ND  {'postcode': 'CR0 1ND', 'quality': 1, 'easting...


(15458, 2)

### Reset index

In [27]:
postcodeWithSchools.set_index(["query"],inplace=True)
postcodeWithSchools.head()

,result
query,
CR0 0EG,"{'postcode': 'CR0 0EG', 'quality': 1, 'easting..."
CR0 0PA,"{'postcode': 'CR0 0PA', 'quality': 1, 'easting..."
CR0 1EQ,"{'postcode': 'CR0 1EQ', 'quality': 1, 'easting..."
CR0 0AH,"{'postcode': 'CR0 0AH', 'quality': 1, 'easting..."
CR0 1ND,"{'postcode': 'CR0 1ND', 'quality': 1, 'easting..."


In [28]:
postcodeWithSchools['result'].replace("\'","\"", regex=True, inplace=True) 
postcodeWithSchools.head(10)


,result
query,
CR0 0EG,"{'postcode': 'CR0 0EG', 'quality': 1, 'easting..."
CR0 0PA,"{'postcode': 'CR0 0PA', 'quality': 1, 'easting..."
CR0 1EQ,"{'postcode': 'CR0 1EQ', 'quality': 1, 'easting..."
CR0 0AH,"{'postcode': 'CR0 0AH', 'quality': 1, 'easting..."
CR0 1ND,"{'postcode': 'CR0 1ND', 'quality': 1, 'easting..."
CR0 1LH,"{'postcode': 'CR0 1LH', 'quality': 1, 'easting..."
CR0 1QH,"{'postcode': 'CR0 1QH', 'quality': 1, 'easting..."
CR0 1XP,"{'postcode': 'CR0 1XP', 'quality': 1, 'easting..."
CR0 2AQ,"{'postcode': 'CR0 2AQ', 'quality': 1, 'easting..."


## Write the result to file for future use

In [29]:
postcodeWithSchools.to_csv(r'./codepo_gb/Data/postcodeWithSchools-clean.csv')

### Read it back

In [30]:
schoolsdf = pd.read_csv('./codepo_gb/Data/postcodeWithSchools-clean.csv')
schoolsdf.head()

,query,result
0,CR0 0EG,"{'postcode': 'CR0 0EG', 'quality': 1, 'easting..."
1,CR0 0PA,"{'postcode': 'CR0 0PA', 'quality': 1, 'easting..."
2,CR0 1EQ,"{'postcode': 'CR0 1EQ', 'quality': 1, 'easting..."
3,CR0 0AH,"{'postcode': 'CR0 0AH', 'quality': 1, 'easting..."
4,CR0 1ND,"{'postcode': 'CR0 1ND', 'quality': 1, 'easting..."


### Evaluate the json dictionary, get the latitude and longitude, add to schools dataframe

In [184]:
from ast import literal_eval

#schooldf1.append(rowdata,ignore_index=True)
d1 = pd.DataFrame(columns=['country','postcode','admin_district','latitude','longitude'])

for i in np.arange(0,len(schoolsdf)):
    #print (i)
    schooljson = schoolsdf.loc[i,'result']
    #schooljson = schooljson.replace("\'","\"") 
    #print(schooljson)

    # use literal_eval to convert to dict
    schooldict = literal_eval(schooljson)
    
    #for k,v in schooldict.items():
    #    print(k,':',v)
    rowdata = {'country':schooldict['country'],'postcode':schooldict['postcode'],'admin_district':schooldict['admin_district'],'latitude':schooldict['latitude'],'longitude':schooldict['longitude']}
    d1 = d1.append(rowdata,ignore_index=True)

print(d1)

        country postcode                       admin_district   latitude  \
0       England  CR0 0EG                              Croydon  51.341824   
1       England  CR0 0PA                              Croydon  51.347706   
2       England  CR0 1EQ                              Croydon  51.365031   
3       England  CR0 0AH                              Croydon  51.334550   
4       England  CR0 1ND                              Croydon  51.370213   
5       England  CR0 1LH                              Croydon  51.374949   
6       England  CR0 1QH                              Croydon  51.368287   
7       England  CR0 1XP                              Croydon  51.374793   
8       England  CR0 2AQ                              Croydon  51.379158   
9       England  CR0 2AL                              Croydon  51.381272   
10      England  CR0 2EA                              Croydon  51.388193   
11      England  CR0 2HN                              Croydon  51.390395   
12      Engl

## Now we have a dataframe with postcodes and latitude and longitude
## Write it to a file

In [32]:
d1.to_csv(r'./codepo_gb/Data/postcodeWithLatitudeAndLongitude.csv')

In [33]:
d1.shape

(15458, 5)

## Now we merge the schools info, latitude and longitude using postcode
## We have a dataframe of schools with latitude and longitude that can be used in Foursquare

In [34]:
# merge dataframes
schoolsWithLatLng = pd.merge(new_df_subset, d1,  how='left', left_on=['Postcode'], right_on = ['postcode'])
schoolsWithLatLng

,EstablishmentTypeGroup (name),EstablishmentName,OfstedRating (name),County (name),Town,Postcode,country,postcode,admin_district,latitude,longitude
0,Local authority maintained schools,Fairchildes Primary School,Outstanding,Surrey,Croydon,CR0 0AH,England,CR0 0AH,Croydon,51.334550,-0.002455
1,Local authority maintained schools,Addington High School,NaN,Surrey,Croydon,CR0 0AH,England,CR0 0AH,Croydon,51.334550,-0.002455
2,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH,England,CR0 0AH,Croydon,51.334550,-0.002455
3,Academies,Fairchildes Primary School,NaN,Surrey,Croydon,CR0 0AH,England,CR0 0AH,Croydon,51.334550,-0.002455
4,Academies,Meridian High School,Requires improvement,Surrey,Croydon,CR0 0AH,England,CR0 0AH,Croydon,51.334550,-0.002455
...,...,...,...,...,...,...,...,...,...,...,...
24014,Special schools,Green Hall Primary School,NaN,Lancashire,Manchester,M46 9HP,England,M46 9HP,Wigan,53.531931,-2.473904
24015,Free Schools,UTC@MediacityUK,Requires improvement,NaN,Salford,M50 2UW,England,M50 2UW,Salford,53.475390,-2.296408
24016,Free Schools,AldridgeUTC@MediaCityUK,NaN,Greater Manchester,Salford,M50 2UW,England,M50 2UW,Salford,53.475390,-2.296408
24017,Academies,Oasis Academy MediaCityUK,Good,NaN,Manchester,M50 3UQ,England,M50 3UQ,Salford,53.474065,-2.282884


## Suppose we want to investigate the City of Bristol

In [143]:
schoolsWithLatLng[schoolsWithLatLng['Town']=="Bristol"]

,EstablishmentTypeGroup (name),EstablishmentName,OfstedRating (name),County (name),Town,Postcode,country,postcode,admin_district,latitude,longitude
6175,Independent schools,Bristol Cathedral School,NaN,NaN,Bristol,BS1 5TS,England,BS1 5TS,"Bristol, City of",51.451211,-2.600784
6176,Academies,Bristol Cathedral Choir School,Outstanding,NaN,Bristol,BS1 5TS,England,BS1 5TS,"Bristol, City of",51.451211,-2.600784
6177,Free Schools,Cathedral Primary School,Good,Bristol,Bristol,BS1 5TS,England,BS1 5TS,"Bristol, City of",51.451211,-2.600784
6178,Colleges,City of Bristol College,Requires improvement,NaN,Bristol,BS1 5UA,England,BS1 5UA,"Bristol, City of",51.450934,-2.605730
6179,Local authority maintained schools,St George Church of England Primary School,Good,NaN,Bristol,BS1 5XJ,England,BS1 5XJ,"Bristol, City of",51.451609,-2.605552
6180,Local authority maintained schools,Redcliffe Childrens Centre and Maintained Nurs...,Outstanding,NaN,Bristol,BS1 6RR,England,BS1 6RR,"Bristol, City of",51.445885,-2.588915
6181,Local authority maintained schools,St Mary Redcliffe and Temple School,Outstanding,NaN,Bristol,BS1 6RT,England,BS1 6RT,"Bristol, City of",51.446007,-2.587695
6182,Independent schools,Andalusia Academy Bristol,Inadequate,Avon,Bristol,BS2 0BA,England,BS2 0BA,"Bristol, City of",51.456626,-2.582605
6183,Local authority maintained schools,Rosemary Nursery School and Children's Centre,Outstanding,NaN,Bristol,BS2 0DT,England,BS2 0DT,"Bristol, City of",51.457825,-2.580188
6184,Local authority maintained schools,Hannah More Primary School,Good,NaN,Bristol,BS2 0LT,England,BS2 0LT,"Bristol, City of",51.453774,-2.579258


### Now import some libraries for create clusters and plots
### The below approach and codes are borrowed from Lab exercise of IBM Data Science Class for Machine Learning

In [144]:
# import libraries

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Find the coordinates of town with name

In [145]:
#address = 'Oxford, Oxfordshire'
address = 'Bristol, England'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oxford are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oxford are 51.4538022, -2.5972985.


### Create a visual map of town
#### Schools are colored according to their Ofsted Rating
#### Good: Blue
#### Outstanding: Green
#### Requires improvement: Red
#### Others: Gray

In [146]:
town = schoolsWithLatLng[schoolsWithLatLng['Town']=="Bristol"]

# create map of Oxford using latitude and longitude values
map_town = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
# {'beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray', 
# 'green', 'lightblue', 'lightgray', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white'}

for lat, lng, schoolName, rating, in zip(town['latitude'], town['longitude'], town['EstablishmentName'], town['OfstedRating (name)']):
    label = '{}, {}, lat:{},lng:{}'.format(schoolName, rating, lat, lng)
    label = folium.Popup(label, parse_html=True)
    if rating == "Good":
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='lightblue',
            fill_opacity=0.7,
            parse_html=False).add_to(map_town)
    elif rating == "Outstanding":
        folium.CircleMarker(
            [lat, lng],
            radius=10,
            popup=label,
            color='green',
            fill=True,
            fill_color='lightgreen',
            fill_opacity=0.7,
            parse_html=False).add_to(map_town)
    elif rating == "Requires improvement":
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color='red',
            fill=True,
            fill_color='lightred',
            fill_opacity=0.7,
            parse_html=False).add_to(map_town)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=2,
            popup=label,
            color='gray',
            fill=True,
            fill_color='lightgray',
            fill_opacity=0.7,
            parse_html=False).add_to(map_town)

    
map_town

### Set up credentials for using Foursquare APIs

In [100]:
CLIENT_ID = 'NJNLFQFQ1JUVRMLQ2LLII5BM2XTUKWLNXEYAF2TGYBSLZRNY' # your Foursquare ID
CLIENT_SECRET = 'FZYI2S3POPHJMIZM0NZVY2KBEEUUSP0FC2YNZO4J2MKNRPGA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NJNLFQFQ1JUVRMLQ2LLII5BM2XTUKWLNXEYAF2TGYBSLZRNY
CLIENT_SECRET:FZYI2S3POPHJMIZM0NZVY2KBEEUUSP0FC2YNZO4J2MKNRPGA


### Suppose we want to explore the neighbourhood of Rosemary Nursery School and Children's Centre which is located in BS2 0DT, we use the API from postcodes.io to get the nearest postcodes

In [147]:
## get nearest postcodes from api.postcodes.io/postcodes/ :postcode /nearest

from requests.utils import requote_uri

# lookup longitude and latitude
postcode='BS2 0DT'

# Webpage url                                                                                                               
url = 'https://api.postcodes.io/postcodes/' + postcode +'/nearest'
urlencoded = requote_uri(url)

# Extract tables
dfneighbourhood = pd.read_json(urlencoded)
print(dfneighbourhood)


   status                                             result
0     200  {'postcode': 'BS2 0DT', 'quality': 1, 'easting...
1     200  {'postcode': 'BS2 9DT', 'quality': 1, 'easting...
2     200  {'postcode': 'BS2 0DB', 'quality': 1, 'easting...
3     200  {'postcode': 'BS2 0DS', 'quality': 1, 'easting...
4     200  {'postcode': 'BS2 0DU', 'quality': 1, 'easting...
5     200  {'postcode': 'BS2 0DQ', 'quality': 1, 'easting...
6     200  {'postcode': 'BS2 0DR', 'quality': 1, 'easting...
7     200  {'postcode': 'BS2 0DE', 'quality': 1, 'easting...
8     200  {'postcode': 'BS2 0DX', 'quality': 1, 'easting...
9     200  {'postcode': 'BS2 9DY', 'quality': 1, 'easting...


In [181]:
dfneighbourhood.loc[1,'result']

{'postcode': 'BS2 9DT',
 'quality': 1,
 'eastings': 359775,
 'northings': 173346,
 'country': 'England',
 'nhs_ha': 'South West',
 'longitude': -2.580344,
 'latitude': 51.457653,
 'european_electoral_region': 'South West',
 'primary_care_trust': 'Bristol',
 'region': 'South West',
 'lsoa': 'Bristol 054D',
 'msoa': 'Bristol 054',
 'incode': '9DT',
 'outcode': 'BS2',
 'parliamentary_constituency': 'Bristol West',
 'admin_district': 'Bristol, City of',
 'parish': 'Bristol, City of, unparished area',
 'admin_county': None,
 'admin_ward': 'Lawrence Hill',
 'ced': None,
 'ccg': 'NHS Bristol, North Somerset and South Gloucestershire',
 'nuts': 'Bristol, City of',
 'codes': {'admin_district': 'E06000023',
  'admin_county': 'E99999999',
  'admin_ward': 'E05010907',
  'parish': 'E43000019',
  'parliamentary_constituency': 'E14000602',
  'ccg': 'E38000222',
  'ccg_id': '15C',
  'ced': 'E99999999',
  'nuts': 'UKK11',
  'lsoa': 'E01033358',
  'msoa': 'E02006887',
  'lau2': 'E05010907'},
 'distance'

In [149]:
print (dfneighbourhood.loc[0,'result']['postcode'],dfneighbourhood.loc[0,'result']['latitude'],dfneighbourhood.loc[0,'result']['longitude'])

BS2 0DT 51.457825 -2.5801879999999997


### We construct a latitude and longitude dataframe by extracting the result from the JSON responses

In [150]:
from ast import literal_eval

#schooldf1.append(rowdata,ignore_index=True)
dfn = pd.DataFrame(columns=['country','Neighbourhood','Latitude','Longitude'])

for i in np.arange(0,len(dfneighbourhood)):
    #print (i)
    dfneighbourhooddict = dfneighbourhood.loc[i,'result']

    # use literal_eval to convert to dict
    #dfneighbourhooddict = json.loads(dfneighbourhoodjson)
    
    #for k,v in dfneighbourhooddict.items():
    #    print(k,':',v)
    
    rowdata = {'country':dfneighbourhooddict['country'],'Neighbourhood':dfneighbourhooddict['postcode'],'Latitude':dfneighbourhooddict['latitude'],'Longitude':dfneighbourhooddict['longitude']}
    dfn = dfn.append(rowdata,ignore_index=True)

print(dfn)

   country Neighbourhood   Latitude  Longitude
0  England       BS2 0DT  51.457825  -2.580188
1  England       BS2 9DT  51.457653  -2.580344
2  England       BS2 0DB  51.457556  -2.579882
3  England       BS2 0DS  51.457786  -2.580720
4  England       BS2 0DU  51.457621  -2.579638
5  England       BS2 0DQ  51.457494  -2.579766
6  England       BS2 0DR  51.457543  -2.580702
7  England       BS2 0DE  51.457402  -2.579968
8  England       BS2 0DX  51.458232  -2.579588
9  England       BS2 9DY  51.458480  -2.580311


### Get neighbourhood details

In [151]:
neighbourhood_latitude = dfn.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = dfn.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = dfn.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of BS2 0DT are 51.457825, -2.5801879999999997.


### Now get the top 100 venues that are in BS2 0DT within a radius of 1000 meters.

In [152]:
neighbourhood_latitude = 51.457825
neighbourhood_longitude = -2.5801879999999997

In [153]:
LIMIT=100
radius=1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NJNLFQFQ1JUVRMLQ2LLII5BM2XTUKWLNXEYAF2TGYBSLZRNY&client_secret=FZYI2S3POPHJMIZM0NZVY2KBEEUUSP0FC2YNZO4J2MKNRPGA&v=20180605&ll=51.457825,-2.5801879999999997&radius=1000&limit=100'

### Returned result set is in json format

In [154]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601822577b18381149062da0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bristol',
  'headerFullLocation': 'Bristol',
  'headerLocationGranularity': 'city',
  'totalResults': 115,
  'suggestedBounds': {'ne': {'lat': 51.46682500900001,
    'lng': -2.5657708256466556},
   'sw': {'lat': 51.44882499099999, 'lng': -2.594605174353344}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5658dfd7498ea630649e7086',
       'name': 'The Old Market Assembly',
       'location': {'address': '25 West Street, Old Market',
        'lat': 51.45678438337498,
        'lng': -2.5790207651872152,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.4

### Helper function for getting the category of the venue

In [155]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [156]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-156-4243e3e1ab9c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,The Old Market Assembly,Bar,51.456784,-2.579021
1,The Volunteer Tavern,Pub,51.457749,-2.582045
2,The Trinity Centre,Music Venue,51.458080,-2.576293
3,25A Old Market,Café,51.455865,-2.582173
4,Cabot Circus,Shopping Mall,51.458507,-2.585154
5,The Barley Mow,Pub,51.453403,-2.577350
6,Harvey Nichols Restaurant & Bar,French Restaurant,51.457233,-2.586228
7,BBB - Bristol Bear Bar,Gay Bar,51.456473,-2.579813
8,Côte Brasserie,French Restaurant,51.457400,-2.587210
9,Nando's,Portuguese Restaurant,51.458234,-2.584539


In [157]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Helper function for getting venues near by a latitude and longitude

In [158]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### We use the new dataframe to get a list of venues for each neighbourhood

In [161]:

nearby_venues = getNearbyVenues(names=dfn['Neighbourhood'],
                                   latitudes=dfn['Latitude'],
                                   longitudes=dfn['Longitude']
                                  )

BS2 0DT
BS2 9DT
BS2 0DB
BS2 0DS
BS2 0DU
BS2 0DQ
BS2 0DR
BS2 0DE
BS2 0DX
BS2 9DY


### We take a look at the shape of the data frames and all the returned venues in Bristol

In [162]:
print(nearby_venues.shape)
nearby_venues

(445, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BS2 0DT,51.457825,-2.580188,The Old Market Assembly,51.456784,-2.579021,Bar
1,BS2 0DT,51.457825,-2.580188,The Volunteer Tavern,51.457749,-2.582045,Pub
2,BS2 0DT,51.457825,-2.580188,25A Old Market,51.455865,-2.582173,Café
3,BS2 0DT,51.457825,-2.580188,The Trinity Centre,51.458080,-2.576293,Music Venue
4,BS2 0DT,51.457825,-2.580188,BBB - Bristol Bear Bar,51.456473,-2.579813,Gay Bar
5,BS2 0DT,51.457825,-2.580188,Cabot Circus,51.458507,-2.585154,Shopping Mall
6,BS2 0DT,51.457825,-2.580188,West Street Kitchen,51.457144,-2.577861,Café
7,BS2 0DT,51.457825,-2.580188,Nando's,51.458234,-2.584539,Portuguese Restaurant
8,BS2 0DT,51.457825,-2.580188,Harvey Nichols Restaurant & Bar,51.457233,-2.586228,French Restaurant
9,BS2 0DT,51.457825,-2.580188,Five Guys,51.458220,-2.584666,Burger Joint


In [163]:
nearby_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
BS2 0DB,44,44,44,44,44,44
BS2 0DE,44,44,44,44,44,44
BS2 0DQ,42,42,42,42,42,42
BS2 0DR,47,47,47,47,47,47
BS2 0DS,48,48,48,48,48,48
BS2 0DT,44,44,44,44,44,44
BS2 0DU,42,42,42,42,42,42
BS2 0DX,42,42,42,42,42,42
BS2 9DT,45,45,45,45,45,45


In [164]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 33 uniques categories.


### Now we create a matrix containing onehot encoding of the venues
### and group them into another dataframe

In [165]:
# one hot encoding
nearby_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nearby_onehot['Neighbourhood'] = nearby_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_onehot.columns[-1]] + list(nearby_onehot.columns[:-1])
nearby_onehot = nearby_onehot[fixed_columns]

nearby_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Chocolate Shop,Clothing Store,Coffee Shop,Department Store,Donut Shop,Electronics Store,Fast Food Restaurant,French Restaurant,Gay Bar,Gift Shop,Greek Restaurant,Gym / Fitness Center,Hotel,Hotel Bar,Italian Restaurant,Multiplex,Music Venue,Park,Pizza Place,Plaza,Portuguese Restaurant,Pub,Shopping Mall,Sushi Restaurant,Wine Bar
0,BS2 0DT,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BS2 0DT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,BS2 0DT,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,BS2 0DT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,BS2 0DT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [166]:
nearby_onehot.shape

(445, 34)

In [167]:
nearby_grouped = nearby_onehot.groupby('Neighbourhood').mean().reset_index()
nearby_grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Chocolate Shop,Clothing Store,Coffee Shop,Department Store,Donut Shop,Electronics Store,Fast Food Restaurant,French Restaurant,Gay Bar,Gift Shop,Greek Restaurant,Gym / Fitness Center,Hotel,Hotel Bar,Italian Restaurant,Multiplex,Music Venue,Park,Pizza Place,Plaza,Portuguese Restaurant,Pub,Shopping Mall,Sushi Restaurant,Wine Bar
0,BS2 0DB,0.022727,0.022727,0.022727,0.022727,0.000000,0.045455,0.022727,0.068182,0.022727,0.068182,0.022727,0.045455,0.022727,0.000000,0.022727,0.022727,0.045455,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.045455,0.000000,0.022727,0.113636,0.022727,0.045455,0.000000
1,BS2 0DE,0.022727,0.022727,0.022727,0.022727,0.000000,0.045455,0.022727,0.068182,0.022727,0.068182,0.022727,0.045455,0.022727,0.000000,0.022727,0.022727,0.045455,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.045455,0.000000,0.022727,0.113636,0.022727,0.045455,0.000000
2,BS2 0DQ,0.023810,0.023810,0.023810,0.023810,0.000000,0.047619,0.023810,0.071429,0.023810,0.047619,0.023810,0.047619,0.023810,0.000000,0.023810,0.023810,0.047619,0.023810,0.023810,0.047619,0.023810,0.023810,0.023810,0.023810,0.047619,0.023810,0.047619,0.000000,0.023810,0.119048,0.023810,0.023810,0.000000
3,BS2 0DR,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.063830,0.021277,0.063830,0.042553,0.042553,0.021277,0.021277,0.021277,0.042553,0.042553,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.042553,0.021277,0.042553,0.000000,0.021277,0.085106,0.021277,0.042553,0.000000
4,BS2 0DS,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.062500,0.020833,0.062500,0.041667,0.041667,0.020833,0.020833,0.020833,0.041667,0.041667,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.041667,0.020833,0.041667,0.020833,0.020833,0.083333,0.020833,0.041667,0.000000
5,BS2 0DT,0.022727,0.022727,0.022727,0.022727,0.000000,0.045455,0.022727,0.068182,0.022727,0.068182,0.022727,0.045455,0.022727,0.000000,0.022727,0.045455,0.045455,0.022727,0.022727,0.045455,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.045455,0.000000,0.022727,0.090909,0.022727,0.045455,0.000000
6,BS2 0DU,0.023810,0.023810,0.023810,0.023810,0.000000,0.047619,0.023810,0.071429,0.023810,0.047619,0.023810,0.047619,0.023810,0.000000,0.023810,0.023810,0.047619,0.023810,0.023810,0.047619,0.023810,0.023810,0.023810,0.023810,0.047619,0.023810,0.047619,0.000000,0.023810,0.119048,0.023810,0.023810,0.000000
7,BS2 0DX,0.023810,0.023810,0.023810,0.023810,0.000000,0.047619,0.023810,0.071429,0.023810,0.047619,0.023810,0.047619,0.023810,0.000000,0.023810,0.023810,0.047619,0.023810,0.023810,0.071429,0.023810,0.023810,0.023810,0.023810,0.047619,0.023810,0.047619,0.000000,0.023810,0.095238,0.023810,0.023810,0.000000
8,BS2 9DT,0.022222,0.022222,0.022222,0.022222,0.000000,0.044444,0.022222,0.066667,0.022222,0.066667,0.022222,0.044444,0.022222,0.022222,0.022222,0.044444,0.044444,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.044444,0.000000,0.022222,0.088889,0.022222,0.044444,0.000000
9,BS2 9DY,0.021277,0.021277,0.021277,0.021277,0.000000,0.042553,0.021277,0.063830,0.021277,0.063830,0.021277,0.042553,0.021277,0.000000,0.021277,0.042553,0.042553,0.021277,0.021277,0.063830,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.042553,0.021277,0.021277,0.085106,0.021277,0.042553,0.021277


In [168]:
nearby_grouped.shape

(10, 34)

In [169]:
num_top_venues = 5

for hood in nearby_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nearby_grouped[nearby_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BS2 0DB----
              venue  freq
0               Pub  0.11
1    Clothing Store  0.07
2              Café  0.07
3       Music Venue  0.05
4  Department Store  0.05


----BS2 0DE----
              venue  freq
0               Pub  0.11
1    Clothing Store  0.07
2              Café  0.07
3       Music Venue  0.05
4  Department Store  0.05


----BS2 0DQ----
              venue  freq
0               Pub  0.12
1              Café  0.07
2           Gay Bar  0.05
3       Music Venue  0.05
4  Department Store  0.05


----BS2 0DR----
                venue  freq
0                 Pub  0.09
1      Clothing Store  0.06
2                Café  0.06
3         Coffee Shop  0.04
4  Italian Restaurant  0.04


----BS2 0DS----
                venue  freq
0                 Pub  0.08
1      Clothing Store  0.06
2                Café  0.06
3         Coffee Shop  0.04
4  Italian Restaurant  0.04


----BS2 0DT----
               venue  freq
0                Pub  0.09
1     Clothing Store  0.07
2        

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [171]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = nearby_grouped['Neighbourhood']

for ind in np.arange(nearby_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nearby_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BS2 0DB,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
1,BS2 0DE,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
2,BS2 0DQ,Pub,Café,Gay Bar,Burger Joint,Department Store,Gym / Fitness Center,Clothing Store,Music Venue,Pizza Place,Fast Food Restaurant
3,BS2 0DR,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue
4,BS2 0DS,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue


### Perform a KMeans clustering and assign a cluster label to each of the grouped neighbourhood

In [173]:
# set number of clusters
kclusters = 5

nearby_grouped_clustering = nearby_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 0, 2, 4, 4, 2, 3], dtype=int32)

In [176]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nearby_merged = dfn

# merge toronto_grouped with df2 to add latitude/longitude for each neighborhood
nearby_merged = nearby_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

nearby_merged.head(10) # check the last columns!

,country,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,England,BS2 0DT,51.457825,-2.580188,2,Pub,Clothing Store,Café,Gay Bar,Music Venue,Sushi Restaurant,Department Store,Gym / Fitness Center,French Restaurant,Burger Joint
1,England,BS2 9DT,51.457653,-2.580344,2,Pub,Clothing Store,Café,Gay Bar,French Restaurant,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue
2,England,BS2 0DB,51.457556,-2.579882,1,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
3,England,BS2 0DS,51.457786,-2.580720,0,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue
4,England,BS2 0DU,51.457621,-2.579638,4,Pub,Café,Gay Bar,Burger Joint,Department Store,Gym / Fitness Center,Clothing Store,Music Venue,Pizza Place,Fast Food Restaurant
5,England,BS2 0DQ,51.457494,-2.579766,4,Pub,Café,Gay Bar,Burger Joint,Department Store,Gym / Fitness Center,Clothing Store,Music Venue,Pizza Place,Fast Food Restaurant
6,England,BS2 0DR,51.457543,-2.580702,0,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue
7,England,BS2 0DE,51.457402,-2.579968,1,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
8,England,BS2 0DX,51.458232,-2.579588,4,Pub,Gym / Fitness Center,Café,Gay Bar,Burger Joint,Department Store,Clothing Store,Music Venue,Pizza Place,Fast Food Restaurant
9,England,BS2 9DY,51.458480,-2.580311,3,Pub,Gym / Fitness Center,Clothing Store,Café,Gay Bar,Music Venue,Sushi Restaurant,Department Store,French Restaurant,Burger Joint


In [177]:
# clean up of data
# drop those items with NAN

nearby_merged.dropna(inplace=True)
nearby_merged = nearby_merged.astype({"Cluster Labels": int})
nearby_merged['Cluster Labels']

0    2
1    2
2    1
3    0
4    4
5    4
6    0
7    1
8    4
9    3
Name: Cluster Labels, dtype: int64

### Create a visual map of clusters in Toronto

In [178]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_merged['Latitude'], nearby_merged['Longitude'], nearby_merged['Neighbourhood'], nearby_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

### Cluster 1

In [179]:
nearby_merged.loc[nearby_merged['Cluster Labels'] == 0, nearby_merged.columns[[1] + list(range(5, nearby_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,BS2 0DS,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue
6,BS2 0DR,Pub,Clothing Store,Café,Gay Bar,Italian Restaurant,French Restaurant,Sushi Restaurant,Coffee Shop,Burger Joint,Music Venue


### Cluster 2

In [180]:
nearby_merged.loc[nearby_merged['Cluster Labels'] == 1, nearby_merged.columns[[1] + list(range(5, nearby_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,BS2 0DB,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
7,BS2 0DE,Pub,Clothing Store,Café,Gay Bar,Sushi Restaurant,Department Store,Pizza Place,Burger Joint,Music Venue,Gym / Fitness Center
